In [1]:
# # if used pip install package
# !pip install xgboost
# !pip install lightgbm
# !pip install wget

In [2]:
import os
import sys
import gc
import wget
import time
import tarfile
import zipfile
import random

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import xgboost as xgb
import lightgbm as lgb

from joblib import Parallel, delayed

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
DDIR = "data"
UDDIR = "user_data"
UFEDIR = "user_data/feat_data_v01"
UMDIR = "user_data/model_data"
RESDIR = "prediction_result"

In [4]:
UID = "user_id"

# Load data (Only once)

In [5]:
# train_fname = wget.download("https://tesla-ap-shanghai-1256322946.cos.ap-shanghai.myqcloud.com/cephfs/tesla_common/deeplearning/dataset/algo_contest/train_preliminary.zip", out=DDIR)
# test_fname = wget.download("https://tesla-ap-shanghai-1256322946.cos.ap-shanghai.myqcloud.com/cephfs/tesla_common/deeplearning/dataset/algo_contest/test.zip", out=DDIR)

In [6]:
# def myunzip(filename):
#     zFile = zipfile.ZipFile(filename, "r")
#     for fileM in zFile.namelist(): 
#         zFile.extract(fileM, DDIR)
#         print(fileM)
#     zFile.close()

In [7]:
# myunzip(train_fname)
# myunzip(test_fname)

# Utils

In [5]:
def bch_rencol(values, prefix="", suffix=""):
    return list(map(lambda x: f"{prefix}"+"_".join(list(map(lambda y: str(y), x)))+f"{suffix}" 
                    if hasattr(x, "__iter__") and not isinstance(x, str) 
                    else f"{prefix}"+str(x)+f"{suffix}", values))

In [6]:
def mynunique(values):
    return values.nunique(dropna=False)

In [7]:
def getidxmax(x):
    return x.idxmax()[1]

In [8]:
OP_SET = ["max", "mean", "std", getidxmax, "nunique"]

# Data Exploration (todo)

In [12]:
# read train data
train_click_log = pd.read_csv(f"{DDIR}/train_preliminary/click_log.csv")
train_ad = pd.read_csv(f"{DDIR}/train_preliminary/ad.csv")
# tag
train_user = pd.read_csv(f"{DDIR}/train_preliminary/user.csv")

In [13]:
# read test data
test_click_log = pd.read_csv(f"{DDIR}/test/click_log.csv")
test_ad = pd.read_csv(f"{DDIR}/test/ad.csv")

In [14]:
train_click_log.shape

(30082771, 4)

In [15]:
train_ad.shape

(2481135, 6)

In [16]:
train_user.shape

(900000, 3)

In [17]:
test_click_log.shape

(33585512, 4)

In [18]:
test_ad.shape

(2618159, 6)

In [19]:
train_ad["product_id"] = train_ad["product_id"].replace("\\N", -1).astype(int)
train_ad["industry"] = train_ad["industry"].replace("\\N", -1).astype(int)

In [20]:
test_ad["product_id"] = test_ad["product_id"].replace("\\N", -1).astype(int)
test_ad["industry"] = test_ad["industry"].replace("\\N", -1).astype(int)

In [21]:
train_ad.count()

creative_id         2481135
ad_id               2481135
product_id          2481135
product_category    2481135
advertiser_id       2481135
industry            2481135
dtype: int64

In [22]:
test_ad.count()

creative_id         2618159
ad_id               2618159
product_id          2618159
product_category    2618159
advertiser_id       2618159
industry            2618159
dtype: int64

# Feature engineering

In [23]:
tol_train = pd.merge(train_click_log, train_ad, how="left", on="creative_id")
tol_test = pd.merge(test_click_log, test_ad, how="left", on="creative_id")

In [24]:
del train_click_log, train_ad
del test_click_log, test_ad

In [25]:
gc.collect()

11

## User Click log (Order 1)

In [ ]:
# for train
tmp = tol_train.groupby([UID], sort=False).agg(
    {
        UID: ["count"], 
        "click_times": ["sum", "max", "mean", "std"],
        "time": ["nunique", "mean", "max", "min"],
        "creative_id": ["nunique"],
        "ad_id": ["nunique"],
        "product_id": ["nunique"],
        "product_category": ["nunique"],
        "advertiser_id": ["nunique"],
        "industry": ["nunique"],
    }
)

In [ ]:
tmp.columns = bch_rencol(tmp.columns)

In [ ]:
tmp.to_pickle(f"{UFEDIR}/train_o1.pkl")
tmp = None

In [ ]:
# for test
tmp = tol_test.groupby([UID], sort=False).agg(
    {
        UID: ["count"], 
        "click_times": ["sum", "max", "mean", "std"],
        "time": ["nunique", "mean", "max", "min"],
        "creative_id": ["nunique"],
        "ad_id": ["nunique"],
        "product_id": ["nunique"],
        "product_category": ["nunique"],
        "advertiser_id": ["nunique"],
        "industry": ["nunique"],
    }
)

In [ ]:
tmp.columns = bch_rencol(tmp.columns)

In [ ]:
tmp.to_pickle(f"{UFEDIR}/test_o1.pkl")
tmp = None

In [ ]:
gc.collect()

## User Click log (Order 2)

In [ ]:
# train
for col in ["creative_id", "ad_id", "product_id", "product_category", "advertiser_id", "industry"]:
    tmp = tol_train.groupby([UID, col], sort=False)[[col]].agg(["count"]).groupby([UID]).agg(OP_SET)
    tmp.columns = bch_rencol(tmp.columns)
    tmp.to_pickle(f"{UFEDIR}/train_o2_{col}.pkl")
    tmp = None

In [ ]:
# test
for col in ["creative_id", "ad_id", "product_id", "product_category", "advertiser_id", "industry"]:
    tmp = tol_test.groupby([UID, col], sort=False)[[col]].agg(["count"]).groupby([UID]).agg(OP_SET)
    tmp.columns = bch_rencol(tmp.columns)
    tmp.to_pickle(f"{UFEDIR}/test_o2_{col}.pkl")
    tmp = None

In [ ]:
gc.collect()

## One-Hot

In [ ]:
# train
tmp = tol_train.groupby([UID, "product_category"], sort=False)[["product_category"]].agg(["count"]).unstack().fillna(0)
tmp.columns = bch_rencol(tmp.columns)

In [ ]:
tmp.to_pickle(f"{UFEDIR}/train_onehot.pkl")
tmp = None

In [ ]:
# test
tmp = tol_test.groupby([UID, "product_category"], sort=False)[["product_category"]].agg(["count"]).unstack().fillna(0)
tmp.columns = bch_rencol(tmp.columns)

In [ ]:
tmp.to_pickle(f"{UFEDIR}/test_onehot.pkl")
tmp = None

In [ ]:
gc.collect()

## Time Windows (Time Bins)

In [ ]:
bins = 9
tol_bins = pd.cut(pd.concat([tol_train["time"], tol_test["time"]]), bins, labels=range(bins))

In [ ]:
tol_train[f"bins{bins}"] = tol_bins[:len(tol_train)]
tol_test[f"bins{bins}"] = tol_bins[len(tol_train):]

### Order 1

In [ ]:
# train
tmp = tol_train.groupby([UID, f"bins{bins}"], sort=False).agg(
    {
        UID: ["count"], 
        "click_times": ["sum", "max", "mean", "std"],
        "time": ["nunique", "mean", "max", "min"],
        "creative_id": ["nunique"],
        "ad_id": ["nunique"],
        "product_id": ["nunique"],
        "product_category": ["nunique"],
        "advertiser_id": ["nunique"],
        "industry": ["nunique"],
    }
).unstack()

In [ ]:
tmp.columns = bch_rencol(tmp.columns, prefix=f"bins{bins}_")

In [ ]:
tmp.to_pickle(f"{UFEDIR}/train_bins{bins}_o1.pkl")
tmp = None

In [ ]:
# test
tmp = tol_test.groupby([UID, f"bins{bins}"], sort=False).agg(
    {
        UID: ["count"], 
        "click_times": ["sum", "max", "mean", "std"],
        "time": ["nunique", "mean", "max", "min"],
        "creative_id": ["nunique"],
        "ad_id": ["nunique"],
        "product_id": ["nunique"],
        "product_category": ["nunique"],
        "advertiser_id": ["nunique"],
        "industry": ["nunique"],
    }
).unstack()

In [ ]:
tmp.columns = bch_rencol(tmp.columns, prefix=f"bins{bins}_")

In [ ]:
tmp.to_pickle(f"{UFEDIR}/test_bins{bins}_o1.pkl")
tmp = None

In [ ]:
gc.collect()

### Order 2

In [ ]:
# space consuming
# # train
# for col in ["creative_id", "ad_id", "product_id", "product_category", "advertiser_id", "industry"]:
#     tmp = tol_train.groupby([UID, f"bins{bins}", col], sort=False)[[col]].agg(["count"]).groupby([UID, f"bins{bins}"]).agg(OP_SET).unstack()
#     tmp.columns = bch_rencol(tmp.columns, prefix=f"bins{bins}_")
#     tmp.to_pickle(f"{UFEDIR}/train_bins{bins}_o2_{col}.pkl")
#     tmp = None

In [ ]:
# space consuming
# # test
# for col in ["creative_id", "ad_id", "product_id", "product_category", "advertiser_id", "industry"]:
#     tmp = tol_test.groupby([UID, f"bins{bins}", col], sort=False)[[col]].agg(["count"]).groupby([UID, f"bins{bins}"]).agg(OP_SET).unstack()
#     tmp.columns = bch_rencol(tmp.columns, prefix=f"bins{bins}_")
#     tmp.to_pickle(f"{UFEDIR}/test_bins{bins}_o2_{col}.pkl")
#     tmp = None

In [ ]:
# train
for col in ["creative_id", "ad_id", "product_id", "product_category", "advertiser_id", "industry"]:
    for i in range(bins):
        tmp = tol_train[tol_train[f"bins{bins}"] == i].groupby([UID, col], sort=False)[[col]].agg(["count"]).groupby([UID]).agg(OP_SET)
        tmp.columns = bch_rencol(tmp.columns, prefix=f"bins{bins}_{i}_")
        tmp.to_pickle(f"{UFEDIR}/train_bins{bins}_{i}_o2_{col}.pkl")
        tmp = None

In [56]:
# test
for col in ["creative_id", "ad_id", "product_id", "product_category", "advertiser_id", "industry"]:
    for i in range(bins):
        tmp = tol_test[tol_test[f"bins{bins}"] == i].groupby([UID, col], sort=False)[[col]].agg(["count"]).groupby([UID]).agg(OP_SET)
        tmp.columns = bch_rencol(tmp.columns, prefix=f"bins{bins}_{i}_")
        tmp.to_pickle(f"{UFEDIR}/test_bins{bins}_{i}_o2_{col}.pkl")
        tmp = None

In [38]:
gc.collect()

37

## Conact

In [26]:
train_feat = pd.DataFrame()
test_feat = pd.DataFrame()
train_feat[UID] = train_user[UID]
test_feat[UID] = np.sort(tol_test[UID].unique())

In [27]:
feat_fname = os.listdir(UFEDIR)

In [28]:
for fname in feat_fname:
    if fname.startswith("train_"):
        train_feat = pd.merge(train_feat, pd.read_pickle(f"{UFEDIR}/{fname}"), how="left", on=UID)
    elif fname.startswith("test_"):
        test_feat = pd.merge(test_feat, pd.read_pickle(f"{UFEDIR}/{fname}"), how="left", on=UID)

In [29]:
# to make sure feat and user(target) have same order
# if true --> sum == 0
np.sum(train_feat[UID] != train_user[UID])

0

In [30]:
train_feat.shape

(900000, 469)

In [31]:
test_feat.shape

(1000000, 469)

In [32]:
del tol_test

In [34]:
train_feat.to_pickle(f"{UDDIR}/train_feat_tol.pkl")
test_feat.to_pickle(f"{UDDIR}/test_feat_tol.pkl")

In [5]:
# train_feat = pd.read_pickle(f"{UDDIR}/train_feat_tol.pkl")
test_feat = pd.read_pickle(f"{UDDIR}/test_feat_tol.pkl")
# train_user = pd.read_csv(f"{DDIR}/train_preliminary/user.csv")

In [50]:
# train_feat["industry_count_getidxmax"] = train_feat["industry_count_getidxmax"].replace("\\N", -1).astype(float)
# train_feat["product_id_count_getidxmax"] = train_feat["product_id_count_getidxmax"].replace("\\N", -1).astype(float)
# test_feat["industry_count_getidxmax"] = test_feat["industry_count_getidxmax"].replace("\\N", -1).astype(float)
# test_feat["product_id_count_getidxmax"] = test_feat["product_id_count_getidxmax"].replace("\\N", -1).astype(float)

# Training&Prediction

In [13]:
# split data
train_feat_tr, train_feat_val, train_tag_tr, train_tag_val = train_test_split(train_feat, train_user, test_size=0.2)

In [14]:
del train_feat, train_user

In [16]:
gc.collect()

0

## For Age

In [25]:
lgbds_train_tr_age = lgb.Dataset(train_feat_tr.drop(UID, axis=1), train_tag_tr["age"]-1)
lgbds_train_val_age = lgb.Dataset(train_feat_val.drop(UID, axis=1), train_tag_val["age"]-1)

In [26]:
params_age = {
    "boosting_type": "gbdt",
    "objective": "multiclass",
    "num_class": 10,
    "metric": "multi_error",
    "learning_rate": 0.1,
}

In [27]:
model_lgb_multi_age = lgb.train(params_age, lgbds_train_tr_age, num_boost_round=1000, valid_sets=[lgbds_train_val_age], verbose_eval=50, early_stopping_rounds=100)

Training until validation scores don't improve for 100 rounds
[50]	valid_0's multi_error: 0.755178
[100]	valid_0's multi_error: 0.743333
[150]	valid_0's multi_error: 0.737283
[200]	valid_0's multi_error: 0.733667
[250]	valid_0's multi_error: 0.731267
[300]	valid_0's multi_error: 0.729544
[350]	valid_0's multi_error: 0.728467
[400]	valid_0's multi_error: 0.728422
[450]	valid_0's multi_error: 0.727561
[500]	valid_0's multi_error: 0.726894
[550]	valid_0's multi_error: 0.726717
[600]	valid_0's multi_error: 0.726294
[650]	valid_0's multi_error: 0.726
[700]	valid_0's multi_error: 0.725694
[750]	valid_0's multi_error: 0.725083
[800]	valid_0's multi_error: 0.725078
[850]	valid_0's multi_error: 0.725394
[900]	valid_0's multi_error: 0.725211
Early stopping, best iteration is:
[821]	valid_0's multi_error: 0.724911


In [ ]:
ndt = time.strftime("%Y%m%d%H%M%S", time.localtime(int(time.time())))
model_lgb_multi_age.save_model(f"{UMDIR}/lgb_multi_age_{ndt}.model")

In [64]:
train_val_age_prob = model_lgb_multi_age.predict(train_feat_val.drop(UID, axis=1), num_iteration=model_lgb_multi_age.best_iteration)

In [65]:
train_val_age_pred = [list(x).index(max(x))+1 for x in train_val_age_prob]

In [66]:
age_acy = accuracy_score(train_val_age_pred, train_tag_val["age"])

In [ ]:
# 20:05-21

## For Gender

In [17]:
lgbds_train_tr_gender = lgb.Dataset(train_feat_tr.drop(UID, axis=1), train_tag_tr["gender"]-1)
lgbds_train_val_gender = lgb.Dataset(train_feat_val.drop(UID, axis=1), train_tag_val["gender"]-1)

In [18]:
params_gender = {
    "boosting_type": "gbdt",
    "objective": "multiclass",
    "num_class": 2,
    "metric": "multi_error",
    "learning_rate": 0.1,
}

In [19]:
model_lgb_multi_gender = lgb.train(params_gender, lgbds_train_tr_gender, num_boost_round=1000, valid_sets=[lgbds_train_val_gender], verbose_eval=50, early_stopping_rounds=100)

Training until validation scores don't improve for 100 rounds
[50]	valid_0's multi_error: 0.242733
[100]	valid_0's multi_error: 0.216417
[150]	valid_0's multi_error: 0.204978
[200]	valid_0's multi_error: 0.197167
[250]	valid_0's multi_error: 0.191839
[300]	valid_0's multi_error: 0.187867
[350]	valid_0's multi_error: 0.185183
[400]	valid_0's multi_error: 0.182867
[450]	valid_0's multi_error: 0.181267
[500]	valid_0's multi_error: 0.180267
[550]	valid_0's multi_error: 0.1794
[600]	valid_0's multi_error: 0.177922
[650]	valid_0's multi_error: 0.177039
[700]	valid_0's multi_error: 0.17615
[750]	valid_0's multi_error: 0.175444
[800]	valid_0's multi_error: 0.174711
[850]	valid_0's multi_error: 0.174178
[900]	valid_0's multi_error: 0.173517
[950]	valid_0's multi_error: 0.172794
[1000]	valid_0's multi_error: 0.1724
Did not meet early stopping. Best iteration is:
[983]	valid_0's multi_error: 0.172378


In [20]:
ndt = time.strftime("%Y%m%d%H%M%S", time.localtime(int(time.time())))
model_lgb_multi_gender.save_model(f"{UMDIR}/lgb_multi_gender_{ndt}.model")

In [21]:
train_val_gender_prob = model_lgb_multi_gender.predict(train_feat_val.drop(UID, axis=1), num_iteration=model_lgb_multi_gender.best_iteration)

In [22]:
train_val_gender_pred = [list(x).index(max(x))+1 for x in train_val_gender_prob]

In [23]:
gender_acy = accuracy_score(train_val_gender_pred, train_tag_val["gender"])

In [24]:
gender_acy

0.8276222222222223

In [67]:
# total accuracy
age_acy + gender_acy

0.9869944444444445

# Generate Prediction Result

In [6]:
model_lgb_multi_age = lgb.Booster(model_file=f"{UMDIR}/lgb_multi_age_20200510133141.model")

In [7]:
model_lgb_multi_gender = lgb.Booster(model_file=f"{UMDIR}/lgb_multi_gender_20200510120440.model")

In [8]:
res = test_feat[[UID]]

In [9]:
test_feat.drop(UID, axis=1, inplace=True)

In [11]:
res["predicted_age"] = [list(x).index(max(x))+1 for x in model_lgb_multi_age.predict(test_feat, num_iteration=model_lgb_multi_age.best_iteration)]

/opt/conda/envs/tensorflow_py3/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [12]:
res["predicted_gender"] = [list(x).index(max(x))+1 for x in model_lgb_multi_gender.predict(test_feat, num_iteration=model_lgb_multi_gender.best_iteration)]

/opt/conda/envs/tensorflow_py3/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [13]:
res_suffix = time.strftime("%Y%m%d%H%M%S", time.localtime(int(time.time())))
res.to_csv(f"{RESDIR}/res-{res_suffix}.csv", index=False)

In [14]:
res

,user_id,predicted_age,predicted_gender
0,3000001,10,2
1,3000002,1,2
2,3000003,10,2
3,3000004,8,2
4,3000005,1,2
...,...,...,...
999995,3999996,10,2
999996,3999997,10,2
999997,3999998,1,2
999998,3999999,1,2


In [10]:
test_feat.shape

(1000000, 468)

# Cent result to COS

In [15]:
from ti import session
ti_session = session.Session()

In [16]:
inputs = ti_session.upload_data(path=f"{RESDIR}/res-20200510140336.csv", bucket="etveritas-1252104022", key_prefix=RESDIR)